In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models
import random

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
import main.seir.uncertainty as uncertainty_module
from utils.generic.create_report import save_dict_and_create_report_simple
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

import yaml

In [ ]:
predictions_dict = {}

In [ ]:
config_filename = 'without_d.yaml'
config = read_config(config_filename)

In [ ]:
output_folder = '../../misc/reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

## Perform M1 and M2 fits

In [ ]:
config_copy = copy.deepcopy(config['fitting'])
print(config_copy['default_params'])
config_copy['default_params']['d'] = 0.02
print(config_copy['default_params'])

In [ ]:
d_vals = [0.01,0.02,0.05,0.1,0.2,0.4,0.5,0.75,0.9,1.0]
# d_vals = [0.01,0.4]
best_params = []
train_losses = []
val_losses = []
for d in d_vals:
    config_copy = copy.deepcopy(config['fitting'])
    config_copy['default_params']['d'] = d
    predictions_dict['m1'] = single_fitting_cycle(**config_copy) 
    predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict['m1']['best_params']['d'] = d
    best_params.append(predictions_dict['m1']['best_params'])
    train_losses.append(predictions_dict['m1']['df_loss']['train'])
    val_losses.append(predictions_dict['m1']['df_loss']['val'])

## Loss Dataframes

In [ ]:
predictions_dict['m1']['df_loss']

In [ ]:
import pandas as pd

df = pd.DataFrame(best_params)
df

In [ ]:
df2 = pd.DataFrame(train_losses)
df2 = df2.add_prefix('trainloss_')
df2 = df2.set_index(df.index)

df3 = pd.DataFrame(val_losses)
df3 = df3.add_prefix('valloss_')
df3 = df3.set_index(df.index)

df2

In [ ]:
df_final = df.join(df2)
df_final = df_final.join(df3)
df_final.to_csv('../../misc/reports/varying_d.csv')

### M2 Loss DataFrame